# Recommend System 의 기본적인 유형
- Content based filtering : A를 좋아하면 A와 비슷한 속성을 지닌 B를 추천
- Collaborative filtering : 평가를 내리지 않은 Item에 대해 어떻게 평가할지 예측
    - 단, K-NN 을 이용한 filtering에서는 User-Item 과 같은 행렬 형태를 지녀야 함(테이블 변환 필요)
    - Item based collaborative filtering를 이용해 Item A, B 평점 분포가 유사한 경우, A, B가 유사하다고 판단하고 평가가 내려지지 않은 Item 이 있으면 이것을 추천하는 것.
- 참고 [추천 시스템 알고리즘](https://lsjsj92.tistory.com/568)

- 필요한 데이터 : 플레이리스트에 속하는 각 노래들과 그 점수(플레이리스트id, 곡id, 좋아요), 곡 정보(곡id, 제목, 장르) 

In [1]:
import pandas as pd
import json
import os

def extract_gnr(group):
    return group['song_gn_dtl_gnr_basket'].tolist()

with open('./../Datasets/song_meta.json', 'rt', encoding='UTF8') as f:
    js = json.loads(f.read())
song_meta = pd.DataFrame(js)
with open('./../Datasets/train.json', 'rt', encoding='UTF8') as f:
    js = json.loads(f.read())
train = pd.DataFrame(js)
train = train.explode("songs")

In [2]:
train_plist = train.drop(['tags','updt_date'], axis = 1)
train_plist.head(10)
#joined = pd.merge(train, song_meta, left_on='songs', right_on='id')
#res = joined.groupby('id_x').apply(extract_gnr)

,id,plylst_title,songs,like_cnt
0,61281,여행같은 음악,525514,71
0,61281,여행같은 음악,129701,71
0,61281,여행같은 음악,383374,71
0,61281,여행같은 음악,562083,71
0,61281,여행같은 음악,297861,71
0,61281,여행같은 음악,139541,71
0,61281,여행같은 음악,351214,71
0,61281,여행같은 음악,650298,71
0,61281,여행같은 음악,531057,71
0,61281,여행같은 음악,205238,71


In [3]:
train_song = song_meta.drop(['album_id', 'album_name', 'artist_id_basket', 'artist_name_basket', 'song_gn_gnr_basket', 'issue_date'], axis = 1)
train_song.head(10)

,song_gn_dtl_gnr_basket,song_name,id
0,[GN0901],Feelings,0
1,"[GN1601, GN1606]","Bach : Partita No. 4 In D Major, BWV 828 - II....",1
2,[GN0901],Solsbury Hill (Remastered 2002),2
3,"[GN1102, GN1101]",Feeling Right (Everything Is Nice) (Feat. Popc...,3
4,"[GN1802, GN1801]",그남자 그여자,4
5,[GN1701],Para Los Enamorados,5
6,"[GN1601, GN1602, GN1614]",Sibelius : Valse Triste Op.44 (시벨리우스 : 슬픈 왈츠 작...,6
7,"[GN1601, GN1602, GN1614]",Superman March (From &#34;Superman&#34; / Live...,7
8,[GN0301],Lovers’ Leap (Feat. Qypthone),8
9,"[GN0105, GN0101]","사랑, 그대라는 멜로디",9


In [4]:
plist_song_rating = pd.merge(train_plist, train_song, left_on = 'songs', right_on = 'id')
plist_song_rating.drop(['id_y'], axis = 1, inplace = True)
plist_song_rating.rename(columns={'id_x':'plist_id'}, inplace = True)
sorted_plist_song_rating = plist_song_rating.sort_values(by='plist_id')
sorted_plist_song_rating.head(10)

,plist_id,plylst_title,songs,like_cnt,song_gn_dtl_gnr_basket,song_name
250865,1,*________* 질리지 않는 이 곡 들!,308020,2,"[GN0303, GN0301]",어깨
5160041,1,*________* 질리지 않는 이 곡 들!,662131,2,"[GN0303, GN0301]",Now I Know (Feat. Joon)
5160038,1,*________* 질리지 않는 이 곡 들!,47805,2,"[GN0302, GN0301]",(Bonus Track) Mic를 잡는 매 순간
5160044,1,*________* 질리지 않는 이 곡 들!,418970,2,"[GN0501, GN0304, GN0505, GN0301]",어이세
5160046,1,*________* 질리지 않는 이 곡 들!,117747,2,"[GN0501, GN0304, GN0505, GN0301]",난장 된지금 (Feat. Snacky Chan)
5267494,2,좋은 꿈을 꿀것같은 사랑스러운 음악,669547,11,"[GN0805, GN0501, GN0502, GN0801, GN0509]",걸리버
2020023,2,좋은 꿈을 꿀것같은 사랑스러운 음악,244008,11,"[GN0508, GN0501, GN0601, GN0503, GN0605]",꿈에 들어와
5064977,2,좋은 꿈을 꿀것같은 사랑스러운 음악,379267,11,"[GN0508, GN0501, GN0502, GN0801, GN0804]",깊어진다 계절이
4297300,2,좋은 꿈을 꿀것같은 사랑스러운 음악,115311,11,[GN0901],Where Are We Now?
1818346,2,좋은 꿈을 꿀것같은 사랑스러운 음악,92908,11,"[GN1701, GN1702]",Misty


# 문제 발생
- 데이터 개수가 너무 많아 pivot table을 생성하지 못하는 문제에 부딪힘
- surprise api 의 SVD 를 이용해 해결해보기로 함
- [참고](https://techblog-history-younghunjo1.tistory.com/117)
- 데이터를 '플레이리스트, 곡, 점수' 로 설정할 것
- 데이터 전처리를 하지 않아 RMSE 가 매우 높은 값을 가짐

In [6]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(plist_song_rating[['plist_id', 'songs', 'like_cnt']], reader = reader)
tr, te = train_test_split(data, test_size=0.25, random_state=42)
algo = SVD()
algo.fit(tr)
pred = algo.test(te)
accuracy.rmse(pred)

RMSE: 928.4936


928.4936282651743

In [9]:
def get_unplayed(train, song_meta, plist_id):
    played_songs = train[train['id'] == plist_id]['songs'].tolist()
    total_songs = song_meta['id'].tolist()
    unplayed_songs = [song for song in total_songs if song not in played_songs]
    print(f'특정 {plist_id}번 플레이리스트가 가진 음악 수 : {len(played_songs)}\n추천한 음악 개수 : {len(unplayed_songs)}\n전체 음악 수 : {len(total_songs)}')
    return unplayed_songs

def recommend_song(algo, plist_id, unplayed_song_list, n=10):
    prediction = [algo.predict(str(plist_id), str(song_id)) for song_id in unplayed_song_list]
    def sort_key(pred):
        return pred.est
    prediction.sort(key=sort_key, reverse=True)
    top_prediction = prediction[:n]
    top_song_id = [int(pred.iid) for pred in top_prediction]
    top_song_rating = [pred.est for pred in top_prediction]
    top_song_title = song_meta[song_meta.id.isin(top_song_id)]['song_name']
    top_song_preds = [(ids, rating, title) for ids, rating, title in zip(top_song_id, top_song_rating, top_song_title)]
    
    return top_song_preds

unplayed_song_list = get_unplayed(train, song_meta, 9)
top_song_preds = recommend_song(algo, 9, unplayed_song_list, n=10)
print()
for top_song in top_song_preds:
    print('* 추천 음악 이름: ', top_song[2])
    print('* 해당 음악의 추천도: ', top_song[1])
    print()

특정 9번 플레이리스트가 가진 음악 수 : 40
추천한 음악 개수 : 707949
전체 음악 수 : 707989

* 추천 음악 이름:  Feelings
* 해당 음악의 추천도:  5

* 추천 음악 이름:  Bach : Partita No. 4 In D Major, BWV 828 - II. Allemande
* 해당 음악의 추천도:  5

* 추천 음악 이름:  Solsbury Hill (Remastered 2002)
* 해당 음악의 추천도:  5

* 추천 음악 이름:  Feeling Right (Everything Is Nice) (Feat. Popcaan & Wale)
* 해당 음악의 추천도:  5

* 추천 음악 이름:  그남자 그여자
* 해당 음악의 추천도:  5

* 추천 음악 이름:  Para Los Enamorados
* 해당 음악의 추천도:  5

* 추천 음악 이름:  Sibelius : Valse Triste Op.44 (시벨리우스 : 슬픈 왈츠 작품번호 44)
* 해당 음악의 추천도:  5

* 추천 음악 이름:  Superman March (From &#34;Superman&#34; / Live At Walt Disney Concert Hall, Los Angeles / 2019)
* 해당 음악의 추천도:  5

* 추천 음악 이름:  Lovers’ Leap (Feat. Qypthone)
* 해당 음악의 추천도:  5

* 추천 음악 이름:  사랑, 그대라는 멜로디
* 해당 음악의 추천도:  5

